In [1]:
from stochastic_volatility_models.src.models.rough_bergomi.rough_bergomi import RoughBergomi
from stochastic_volatility_models.src.models.heston.heston import Heston
from notebook_utils import Notebook

nb = Notebook()

h_parameters = {
	"initial_variance": 0.10006713187966065,
	"long_term_variance": 0.03159120049914011,
	"volatility_of_volatility": 3.4949236489765254,
	"mean_reversion_rate": 6.106005002941104,
	"wiener_correlation": -0.7977506958223608,
}

r_parameters = {
	"hurst_index": 0.8000001750483351,
	"wiener_correlation": -0.9979998892239947,
	"volatility_of_volatility": 2.339999541644233,
}

h_nb = Notebook(Heston(h_parameters))
r_nb = Notebook(RoughBergomi(r_parameters))

2024-08-20 12:33:33.725 | INFO     | stochastic_volatility_models.config:initialise:30 - Loaded module `stochastic_volatility_models` from project directory path `/Users/mayurankv/Documents/Mayuran/Programming/Projects/Academic/Imperial College London/MSc Statistics/Dissertation/Project/modules`


In [2]:
import numpy as np
from stochastic_volatility_models.src.core.model import pricing_variance_paths
from stochastic_volatility_models.src.utils.options.expiry import time_to_expiry

steps_per_year = 365

strikes = r_nb.joint_volatility_surface.volatility_strikes
strikes = np.array([30, 32.5, 35, 37.5, 40])
time_to_expiries = time_to_expiry(
	time=r_nb.time,
	option_expiries=r_nb.joint_volatility_surface.volatility_expiries,
)
time_to_expiries = np.array([0.0739726] * len(strikes))
flags = np.array([1] * len(strikes))

price_process, variance_process = r_nb.paths(
	steps_per_year=steps_per_year,
	num_paths=2**15,
)
assert variance_process is not None
prices = pricing_variance_paths(
	variance_process=variance_process,
	flags=flags,
	strikes=strikes,
	time_to_expiries=time_to_expiries,
	steps_per_year=steps_per_year,
)

VIX_WINDOW = 1 / 12
dt = 1 / steps_per_year

flag = 1
spot = nb.volatility_underlying.price(time=nb.time)
t2x = 0.0547945205479452
num_paths = 2**15
strike = spot

e = (
	100
	* np.sqrt(
		np.sum(
			variance_process[
				:,
				int(t2x * steps_per_year) : int((t2x + VIX_WINDOW) * steps_per_year),
			],
			axis=1,
		)
		/ VIX_WINDOW
	)
	* dt
	/ VIX_WINDOW
)
# - strikes[idx]
p = np.maximum((e - strike) * flag, 0)
ap = np.mean(p)
print(ap)

2.8522608359778996


In [3]:
test_nb = Notebook(
	Heston(
		{
			"initial_variance": 0,
			"long_term_variance": 0,
			"volatility_of_volatility": 3.359804039034719,
			"mean_reversion_rate": 5.729368933401853,
			"wiener_correlation": -0.6771983326245793,
		}
	)
)
nb.plot_surfaces(volatility=True)
test_nb.plot_surfaces(volatility=True)
nb.plot_joint_price(volatility=True)
test_nb.plot_joint_price(volatility=True)
nb.plot_iv(volatility=True)
test_nb.plot_iv(volatility=True)

In [7]:
test_nb.evaluate_fit(0, 0.1)

0.42708981709414817

In [5]:
nb.plot_surfaces(volatility=False)
test_nb.plot_surfaces(volatility=False)
nb.plot_joint_price(volatility=False)
test_nb.plot_joint_price(volatility=False)
nb.plot_iv(volatility=False)
test_nb.plot_iv(volatility=False)